In [4]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [7]:
CENSUS_TRACTS_GEOJSON = "raw/census_tracts.geojson"   # e.g., 2020 Census tracts GeoJSON/GeoPackage/SHAPE
PANTRIES_PATH      = "data/pantries.csv"    # can be a GeoJSON of points OR a CSV with lat/lon columns
OPTIONAL_NEIGHBORHOODS_PATH = None                      # e.g., neighborhoods or NTA boundaries (GeoJSON), or None
SUPPLY_GAP_CSV     = "data/Emergency_Food_Supply_Gap_20251110.csv"                    # e.g., "data/supply_gap_by_neighborhood.csv" or None

In [17]:
tracts = gpd.read_file(CENSUS_TRACTS_GEOJSON)  # or your cleaned tract GeoJSON
# make sure it has at least ['geoid','nta2020','ntaname']
pantries = pd.read_csv(PANTRIES_PATH)
metrics = pd.read_csv(SUPPLY_GAP_CSV)

# 3️⃣ Turn pantries into a GeoDataFrame
gpantries = gpd.GeoDataFrame(
    pantries,
    geometry=[Point(xy) for xy in zip(pantries["lng"], pantries["lat"])],
    crs="EPSG:4326"   # WGS84 coordinates
)

# 4️⃣ Ensure coordinate systems match
tracts = tracts.to_crs(epsg=4326)

In [18]:
pantry_in_tract = gpd.sjoin(gpantries, tracts[["geoid","nta2020","ntaname","geometry"]],
                            how="left", predicate="within")

In [19]:
tract_pantry_counts = (
    pantry_in_tract.groupby("geoid")
    .size()
    .reset_index(name="pantry_count")
)


In [20]:
metrics = metrics.rename(
    columns={"Neighborhood Tabulation Area NTA)": "nta2020"}
)
metrics["nta2020"] = metrics["nta2020"].str.strip().str.upper()

tracts = tracts.merge(metrics, on="nta2020", how="left")

In [22]:
tracts = tracts.merge(tract_pantry_counts, on="geoid", how="left")
tracts["pantry_count"] = tracts["pantry_count"].fillna(0)

In [23]:
nta_pantry_counts = (
    pantry_in_tract.groupby(["nta2020","ntaname"])
    .size()
    .reset_index(name="pantries_per_nta")
)

In [25]:
nta_pantry_counts

,nta2020,ntaname,pantries_per_nta
0,BK0101,Greenpoint,1
1,BK0102,Williamsburg,1
2,BK0103,South Williamsburg,1
3,BK0104,East Williamsburg,1
4,BK0202,Downtown Brooklyn-DUMBO-Boerum Hill,3
...,...,...,...
149,SI0201,Grasmere-Arrochar-South Beach-Dongan Hills,2
150,SI0202,New Dorp-Midland Beach,1
151,SI0203,Todt Hill-Emerson Hill-Lighthouse Hill-Manor H...,1
152,SI0302,Great Kills-Eltingville,1


In [27]:
tracts.to_file("data/tracts_with_pantry_counts.geojson", driver="GeoJSON")
nta_pantry_counts.to_csv("data/pantries_per_neighborhood.csv", index=False)

print(nta_pantry_counts.sort_values("pantries_per_nta", ascending=False))

    nta2020                                      ntaname  pantries_per_nta
8    BK0301                    Bedford-Stuyvesant (West)                12
13   BK0502                        East New York (North)                12
99   MN1002                               Harlem (North)                11
135  QN1205                                   St. Albans                10
131  QN1201                                      Jamaica                 9
..      ...                                          ...               ...
123  QN0901                                  Kew Gardens                 1
37   BK1502                                      Madison                 1
92   MN0801  Upper East Side-Lenox Hill-Roosevelt Island                 1
126  QN0904                           Ozone Park (North)                 1
0    BK0101                                   Greenpoint                 1

[154 rows x 3 columns]
